In [1]:
class TreeNode:
    def __init__(self, attribute=None, value=None, left=None, right=None, label=None):
        self.attribute = attribute
        self.value = value
        self.left = left
        self.right = right
        self.label = label

class AttributeList:
    def __init__(self, attribute, values, classes):
        self.attribute = attribute
        self.values = values
        self.classes = classes    

In [2]:
def read_data(file_path):
    # Read the input dataset from a CSV file, preprocess, and return the dataset
    dataset = {}

    with open(file_path, 'r') as file:
        lines = file.read().strip().split('\n')

    # Extract attribute names from the first line
    attribute_names = lines[0].split(',')

    # Initialize empty lists for each attribute
    for attribute in attribute_names:
        dataset[attribute] = []

    # Iterate through the remaining lines and populate the dataset
    for line in lines[1:]:
        values = line.split(',')

        for index, value in enumerate(values):
            attribute = attribute_names[index]

            # Convert numeric values to int or float
            if value.isdigit():
                value = int(value)
            elif value.replace('.', '').isdigit():
                value = float(value)

            dataset[attribute].append(value)

    return dataset

In [3]:
def discretize(dataset, continuous_attributes, num_intervals=5):
    """
    Discretize continuous attributes in the dataset using equal-width binning.

    :param dataset: The input dataset as a dictionary with attribute names as keys and lists of values as values.
    :param continuous_attributes: A list of continuous attribute names.
    :param num_intervals: The number of intervals for equal-width binning (default: 5).
    """
    for attribute in continuous_attributes:
        # Find the range (min and max values) of the continuous attribute
        min_value = min(dataset[attribute])
        max_value = max(dataset[attribute])

        # Calculate the width of each interval
        interval_width = (max_value - min_value) / num_intervals

        # Discretize the continuous attribute into equal-width intervals
        discretized_values = []
        for value in dataset[attribute]:
            interval_index = int((value - min_value) / interval_width)
            interval_start = min_value + interval_index * interval_width
            interval_end = interval_start + interval_width
            discretized_values.append(f'{interval_start:.2f}-{interval_end:.2f}')

        # Replace the continuous attribute values with the corresponding discretized values
        dataset[attribute] = discretized_values

In [4]:
def initialize_attribute_lists(dataset, target_attribute):
    """
    Initialize attribute lists for each attribute in the dataset.

    :param dataset: The input dataset as a dictionary with attribute names as keys and lists of values as values.
    :param target_attribute: The name of the target attribute (class label).
    :return: A dictionary with attribute names as keys and AttributeList objects as values.
    """
    attribute_lists = {}

    # Get the sorted indices of the target attribute values
    sorted_indices = sorted(range(len(dataset[target_attribute])), key=lambda i: dataset[target_attribute][i])

    for attribute in dataset:
        if attribute != target_attribute:
            # Initialize the attribute list for the current attribute
            values = [dataset[attribute][i] for i in sorted_indices]
            classes = [dataset[target_attribute][i] for i in sorted_indices]
            attribute_lists[attribute] = AttributeList(attribute, values, classes)

    return attribute_lists

In [5]:
def gini_coefficient(x):
    """
    Calculate the Gini coefficient for a given list of values.

    :param x: A list of values.
    :return: The Gini coefficient.
    """
    n = len(x)
    sorted_x = sorted(x)
    total = sum(abs(sorted_x[i] - sorted_x[j]) for i in range(n) for j in range(i + 1, n))
    return total / (n * n * sum(x) / n)

def gini_impurity(class_counts):
    """
    Calculate the Gini impurity index for a given class count distribution.

    :param class_counts: A list of class counts.
    :return: The Gini impurity index.
    """
    total_count = sum(class_counts)
    gini = 1.0
    for count in class_counts:
        prob = count / total_count
        gini -= prob ** 2
    return gini

In [6]:
def find_best_split(attribute_list):
    """
    Find the best split for the given attribute list based on the Gini index.

    :param attribute_list: An AttributeList object containing the attribute values and corresponding class labels.
    :return: A tuple containing the best split value and the corresponding Gini index.
    """
    best_split_value = None
    best_gini = float('inf')

    # Initialize class count dictionaries for left and right partitions
    left_counts = {}
    right_counts = {cls: 0 for cls in set(attribute_list.classes)}

    # Count the total number of instances for each class in the right partition
    for cls in attribute_list.classes:
        right_counts[cls] += 1

    # Iterate through the unique attribute values to find the best split
    unique_values = sorted(set(attribute_list.values))
    for i in range(len(unique_values) - 1):
        split_value = unique_values[i]
        split_index = attribute_list.values.index(split_value)

        # Update the class count dictionaries for left and right partitions
        for j in range(split_index + 1):
            cls = attribute_list.classes[j]
            left_counts[cls] = left_counts.get(cls, 0) + 1
            right_counts[cls] -= 1

        # Calculate the weighted Gini index for the current split
        left_total = sum(left_counts.values())
        right_total = sum(right_counts.values())
        gini = (left_total * gini_coefficient(list(left_counts.values())) +
                right_total * gini_coefficient(list(right_counts.values()))) / (left_total + right_total)

        # Update the best split if the current split has a lower Gini index
        if gini < best_gini:
            best_split_value = split_value
            best_gini = gini

    return best_split_value, best_gini

def majority_label(class_labels):
    """
    Find the majority label in a given list of class labels.

    :param class_labels: A list of class labels.
    :return: The majority label.
    """
    label_counts = {}
    for label in class_labels:
        label_counts[label] = label_counts.get(label, 0) + 1

    return max(label_counts, key=label_counts.get)

def split_attribute_list(attribute_list, split_value):
    """
    Split an AttributeList object based on a given split value.

    :param attribute_list: An AttributeList object containing the attribute values and corresponding class labels.
    :param split_value: The value to split the attribute list.
    :return: Two AttributeList objects representing the left and right partitions.
    """
    left_values = []
    left_classes = []
    right_values = []
    right_classes = []

    for value, cls in zip(attribute_list.values, attribute_list.classes):
        if value == split_value:
            left_values.append(value)
            left_classes.append(cls)
        else:
            right_values.append(value)
            right_classes.append(cls)

    left_partition = AttributeList(attribute_list.attribute, left_values, left_classes)
    right_partition = AttributeList(attribute_list.attribute, right_values, right_classes)

    return left_partition, right_partition

def grow_tree(attribute_lists, min_leaf_size=5):
    """
    Grow the decision tree recursively using the best splits found in the attribute_lists.

    :param attribute_lists: A dictionary with attribute names as keys and AttributeList objects as values.
    :param min_leaf_size: The minimum number of instances required to split a node (default: 5).
    :return: The root node of the decision tree.
    """
    # Base case: if the attribute_lists are empty or all instances have the same class label, create a leaf node
    if not attribute_lists or len(set(attribute_lists[next(iter(attribute_lists))].classes)) == 1:
        return TreeNode(label=majority_label(attribute_lists[next(iter(attribute_lists))].classes))

    # Find the best attribute and split value based on the Gini index
    best_attribute = None
    best_split_value = None
    best_gini = float('inf')
    for attribute in attribute_lists:
        split_value, gini = find_best_split(attribute_lists[attribute])
        if gini < best_gini:
            best_attribute = attribute
            best_split_value = split_value
            best_gini = gini

    # If the best split doesn't satisfy the minimum leaf size criterion, create a leaf node
    if sum(value == best_split_value for value in attribute_lists[best_attribute].values) < min_leaf_size:
        return TreeNode(label=majority_label(attribute_lists[best_attribute].classes))

    # Split the attribute lists based on the best attribute and split value
    left_attribute_lists = {}
    right_attribute_lists = {}
    for attribute in attribute_lists:
        left_partition, right_partition = split_attribute_list(attribute_lists[attribute], best_split_value)
        left_attribute_lists[attribute] = left_partition
        right_attribute_lists[attribute] = right_partition

    # Recursively grow the left and right subtrees
    left_child = grow_tree(left_attribute_lists, min_leaf_size)
    right_child = grow_tree(right_attribute_lists, min_leaf_size)

    # Create a tree node with the best attribute and split value
    return TreeNode(attribute=best_attribute, value=best_split_value, left=left_child, right=right_child)

def prune_tree(tree, alpha=0.5):
    """
    Prune the decision tree using a minimum description length (MDL) based approach.

    :param tree: The root node of the decision tree.
    :param alpha: The regularization parameter for the MDL criterion (default: 0.5).
    :return: The pruned decision tree.
    """
    if not tree.left or not tree.right:
        return tree

    # Prune the left and right subtrees recursively
    tree.left = prune_tree(tree.left, alpha)
    tree.right = prune_tree(tree.right, alpha)

    # Calculate the MDL criterion for the current node
    mdl = alpha * (len(tree.left.label) + len(tree.right.label)) + (1 - alpha) * (gini_coefficient(tree.label))

    # If pruning the current node results in a lower MDL criterion, replace it with a leaf node
    if mdl < gini_coefficient(tree.label):
        tree.attribute = None
        tree.value = None
        tree.left = None
        tree.right = None
        tree.label = majority_label(tree.label)

    return tree

def sliq(file_path):
    # Main function to execute the SLIQ algorithm
    dataset = read_data(file_path)
    continuous_attributes = ["Applicant_Income", "Coapplicant_Income", "Loan_Amount"]
    discretize(dataset, continuous_attributes, num_intervals=5)
    attribute_lists = initialize_attribute_lists(dataset, target_attribute="Credit_History")
    tree = grow_tree(attribute_lists)
    pruned_tree = prune_tree(tree)
    return pruned_tree

sliq('loan_train.csv');

In [7]:
def majority_label(class_labels):
    """
    Find the majority label in a given list of class labels.

    :param class_labels: A list of class labels.
    :return: The majority label.
    """
    label_counts = {}
    for label in class_labels:
        label_counts[label] = label_counts.get(label, 0) + 1

    return max(label_counts, key=label_counts.get)

def split_attribute_list(attribute_list, split_value):
    """
    Split an AttributeList object based on a given split value.

    :param attribute_list: An AttributeList object containing the attribute values and corresponding class labels.
    :param split_value: The value to split the attribute list.
    :return: Two AttributeList objects representing the left and right partitions.
    """
    left_values = []
    left_classes = []
    right_values = []
    right_classes = []

    for value, cls in zip(attribute_list.values, attribute_list.classes):
        if value == split_value:
            left_values.append(value)
            left_classes.append(cls)
        else:
            right_values.append(value)
            right_classes.append(cls)

    left_partition = AttributeList(attribute_list.attribute, left_values, left_classes)
    right_partition = AttributeList(attribute_list.attribute, right_values, right_classes)

    return left_partition, right_partition

def grow_tree(attribute_lists, min_leaf_size=5):
    """
    Grow the decision tree recursively using the best splits found in the attribute_lists.

    :param attribute_lists: A dictionary with attribute names as keys and AttributeList objects as values.
    :param min_leaf_size: The minimum number of instances required to split a node (default: 5).
    :return: The root node of the decision tree.
    """
    # Base case: if the attribute_lists are empty or all instances have the same class label, create a leaf node
    if not attribute_lists or len(set(attribute_lists[next(iter(attribute_lists))].classes)) == 1:
        return TreeNode(label=majority_label(attribute_lists[next(iter(attribute_lists))].classes))

    # Find the best attribute and split value based on the Gini index
    best_attribute = None
    best_split_value = None
    best_gini = float('inf')
    for attribute in attribute_lists:
        split_value, gini = find_best_split(attribute_lists[attribute])
        if gini < best_gini:
            best_attribute = attribute
            best_split_value = split_value
            best_gini = gini

    # If the best split doesn't satisfy the minimum leaf size criterion, create a leaf node
    if sum(value == best_split_value for value in attribute_lists[best_attribute].values) < min_leaf_size:
        return TreeNode(label=majority_label(attribute_lists[best_attribute].classes))

    # Split the attribute lists based on the best attribute and split value
    left_attribute_lists = {}
    right_attribute_lists = {}
    for attribute in attribute_lists:
        left_partition, right_partition = split_attribute_list(attribute_lists[attribute], best_split_value)
        left_attribute_lists[attribute] = left_partition
        right_attribute_lists[attribute] = right_partition

    # Recursively grow the left and right subtrees
    left_child = grow_tree(left_attribute_lists, min_leaf_size)
    right_child = grow_tree(right_attribute_lists, min_leaf_size)

    # Create a tree node with the best attribute and split value
    return TreeNode(attribute=best_attribute, value=best_split_value, left=left_child, right=right_child)

def prune_tree(tree, alpha=0.5):
    """
    Prune the decision tree using a minimum description length (MDL) based approach.

    :param tree: The root node of the decision tree.
    :param alpha: The regularization parameter for the MDL criterion (default: 0.5).
    :return: The pruned decision tree.
    """
    if not tree.left or not tree.right:
        return tree

    # Prune the left and right subtrees recursively
    tree.left = prune_tree(tree.left, alpha)
    tree.right = prune_tree(tree.right, alpha)

    # Calculate the MDL criterion for the current node
    mdl = alpha * (len(tree.left.label) + len(tree.right.label)) + (1 - alpha) * (gini_coefficient(tree.label))

    # If pruning the current node results in a lower MDL criterion, replace it with a leaf node
    if mdl < gini_coefficient(tree.label):
        tree.attribute = None
        tree.value = None
        tree.left = None
        tree.right = None
        tree.label = majority_label(tree.label)

    return tree

def sliq(file_path):
    # Main function to execute the SLIQ algorithm
    dataset = read_data(file_path)
    continuous_attributes = ["Applicant_Income", "Coapplicant_Income", "Loan_Amount"]
    discretize(dataset, continuous_attributes, num_intervals=5)
    attribute_lists = initialize_attribute_lists(dataset, target_attribute="Credit_History")
    tree = grow_tree(attribute_lists)
    pruned_tree = prune_tree(tree)
    return pruned_tree

sliq('loan_train.csv');